In [23]:
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords

In [24]:
stopwords_list = stopwords.words('russian')
listToAdd = ['который', 'весь', 'это', 'ваш', 'такой', 'свой', 'наш', 'самый', 'никто', 'однако', 'наверное', 'вскоре', 'сей']
stopwords_list.extend(listToAdd)
stopwords = set(stopwords_list)

In [25]:
morph_1 = Mystem()
def lemmatize_Mystem(text):
    tokens = morph_1.lemmatize(text)
    tokens = [token for token in tokens if token not in stopwords]
    return ''.join(tokens).strip()

In [4]:
# функция парсинга текста
def text_parser(url):
    try:
        html = urlopen(url).read().decode('utf-8')
        pattern_2 = r'.+"text":"(.+)"}</script>'
        text = re.findall(pattern_2, html)[0].replace(u'\xa0', ' ')
    except:
        text = ''
    return text

In [5]:
def text_cleaner(link):
    text = text_parser(link)
    text = re.sub(r'[._^%$#!~@,-«»]+', ' ', text)
    text = re.sub('[^а-яА-Я ]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.lower()
# на выходе получаем русский текст без знаков препинания в нижнем регистре

In [6]:
tags_list = ['находка', 'литература', 'психология', 'наука', 'россия']

In [7]:
df = pd.read_csv('1_filtered_df.csv')

In [8]:
df.head()

,id,time,likes,comments,reposts,views,text,ctr,url,year,month,day,tags
0,320900,1605604268,57,3,3,7159,Мужчина ухаживает за животными уже 25 лет. Де...,0.80,https://knife.media/raccoon-whisperer/,2020,2020-11,2020-11-17,"['животные', 'видео', 'люди', 'находка']"
1,320899,1605604203,33,1,5,7926,"Знаете, как добывали подноготную правду, зачем...",0.42,https://knife.media/origin-of-idioms/,2020,2020-11,2020-11-17,"['языки', 'история', 'насилие', 'россия']"
2,320894,1605600004,22,0,2,7755,"От проектов, помогающих детям-сиротам, до IT-с...",0.28,https://knife.media/edu-projects/,2020,2020-11,2020-11-17,"['партнерский материал', 'образование', 'будущ..."
3,320828,1605546303,106,0,13,14448,"Сказки братьев Гримм знают и любят и все, хотя...",0.73,https://knife.media/bratya-grimm/,2020,2020-11,2020-11-16,"['книги', 'литература', 'люди', 'россия']"
4,320815,1605544203,182,30,22,28388,"Раса, пол и гендер для дзен-буддистов — пустот...",0.64,https://knife.media/queer-zen/,2020,2020-11,2020-11-16,"['ЛГБТ', 'религия']"


In [9]:
from ast import literal_eval
df.tags = df.tags.apply(literal_eval)

In [10]:
explode_tags = df.explode('tags')

In [11]:
df_for_classification = explode_tags.query("tags in @ tags_list")
df_for_classification.head()

,id,time,likes,comments,reposts,views,text,ctr,url,year,month,day,tags
0,320900,1605604268,57,3,3,7159,Мужчина ухаживает за животными уже 25 лет. Де...,0.80,https://knife.media/raccoon-whisperer/,2020,2020-11,2020-11-17,находка
1,320899,1605604203,33,1,5,7926,"Знаете, как добывали подноготную правду, зачем...",0.42,https://knife.media/origin-of-idioms/,2020,2020-11,2020-11-17,россия
2,320894,1605600004,22,0,2,7755,"От проектов, помогающих детям-сиротам, до IT-с...",0.28,https://knife.media/edu-projects/,2020,2020-11,2020-11-17,россия
3,320828,1605546303,106,0,13,14448,"Сказки братьев Гримм знают и любят и все, хотя...",0.73,https://knife.media/bratya-grimm/,2020,2020-11,2020-11-16,литература
3,320828,1605546303,106,0,13,14448,"Сказки братьев Гримм знают и любят и все, хотя...",0.73,https://knife.media/bratya-grimm/,2020,2020-11,2020-11-16,россия


In [12]:
df_for_classification.shape

(5196, 13)

In [13]:
print(f'Количество статей {df_for_classification.id.nunique()}')

Количество статей 4379


In [14]:
df_for_classification_target = pd.get_dummies(df_for_classification[['id', 'tags']]).groupby('id', as_index=False).sum()
df_for_classification_target.head()

,id,tags_литература,tags_наука,tags_находка,tags_психология,tags_россия
0,145781,0,0,0,1,0
1,145784,0,1,0,0,0
2,145792,0,1,0,0,0
3,145804,0,1,0,0,0
4,145835,0,1,0,0,0


In [15]:
# в скольких статьях встречается каждый тег
df_for_classification_target.iloc[::, 1:].sum()

tags_литература     719
tags_наука          890
tags_находка        501
tags_психология     869
tags_россия        2217
dtype: int64

In [16]:
df_for_classification_feature = df.query("id in @df_for_classification_target.id")[['id', 'ctr', 'url']]
df_for_classification_feature.head()

,id,ctr,url
0,320900,0.80,https://knife.media/raccoon-whisperer/
1,320899,0.42,https://knife.media/origin-of-idioms/
2,320894,0.28,https://knife.media/edu-projects/
3,320828,0.73,https://knife.media/bratya-grimm/
5,320795,0.71,https://knife.media/quantum-brain/


In [17]:
df_for_classification_feature.shape

(4379, 3)

In [18]:
df_for_classification_feature['text'] = df_for_classification_feature.url.apply(text_cleaner)

,id,ctr,url,text
0,320900,0.80,https://knife.media/raccoon-whisperer/,наша копилка контента на ютубе пополнилась еще...
1,320899,0.42,https://knife.media/origin-of-idioms/,одеться шиворот навыворот встать как вкопанно...
2,320894,0.28,https://knife.media/edu-projects/,государственная система образования всегда кон...
3,320828,0.73,https://knife.media/bratya-grimm/,якоб и вильгельм гриммы романтики и основополо...
5,320795,0.71,https://knife.media/quantum-brain/,изучая многочисленные концепции сознания вы ра...


In [ ]:
df_for_classification_feature['text_lemm'] = df.text.apply(lemmatize_Mystem)

In [ ]:
df_for_classification = df_for_classification_feature.merge(df_for_classification_target, on='id', how='inner')

In [ ]:
df_for_classification.shape

In [ ]:
df_for_classification.isna().sum()

In [30]:
df_for_classification.dropna(inplace=True)

In [ ]:
df_for_classification.shape

In [ ]:
df_for_classification.to_csv('4_for_classification.csv', index=False)